# Prédiction et Validation du Modèle d'Hospitalisation

Ce notebook combine la construction du modèle de prédiction et sa validation approfondie pour détecter le surapprentissage.

In [ ]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from notebooks.model_validation import (
    plot_learning_curves,
    evaluate_cross_validation,
    plot_prediction_errors,
    calculate_error_statistics
)

## 1. Chargement et Préparation des Données

In [ ]:
# Chargement des données
df_hospi_dep_ens = pd.read_csv('../data/processed/hospi_dep_ens.csv')
df_hospi_dep_gen = pd.read_csv('../data/processed/hospi_dep_gen.csv')
df_hospi_reg_ens = pd.read_csv('../data/processed/hospi_reg_ens.csv')
df_hospi_reg_gen = pd.read_csv('../data/processed/hospi_reg_gen.csv')

# Création des données annuelles pour le niveau régional
df_hospi_reg_ens_yr = df_hospi_reg_ens.groupby(['reg', 'year']).agg({
    'nbr_hospi': 'sum',
    'pop': 'mean',
    'tx_hospi': 'mean',
    'part_femmes': 'mean',
    'part_hommes': 'mean',
    'age_moyen': 'mean'
}).reset_index()

print("Shape du DataFrame:", df_hospi_reg_ens_yr.shape)
df_hospi_reg_ens_yr.head()

## 2. Configuration et Entraînement du Modèle avec PyCaret

In [ ]:
# Initialisation de PyCaret
reg_setup = setup(
    data=df_hospi_reg_ens_yr,
    target='nbr_hospi',
    train_size=0.8,
    session_id=123,
    silent=True,
    fold=5
)

# Comparaison des modèles
best_model = compare_models(n_select=1)

# Création du modèle final
final_model = create_model(best_model)

## 3. Validation Approfondie du Modèle

In [ ]:
# Obtention des données d'entraînement
X = get_config('X')
y = get_config('y')

# 1. Courbes d'apprentissage
print("Traçage des courbes d'apprentissage...")
plot_learning_curves(final_model, X, y)

# 2. Validation croisée détaillée
print("\nÉvaluation de la validation croisée...")
scores, mae_scores, rmse_scores = evaluate_cross_validation(final_model, X, y)

# 3. Prédictions et analyse des erreurs
predictions = predict_model(final_model)
y_true = predictions['nbr_hospi']
y_pred = predictions['prediction_label']

print("\nAnalyse des erreurs de prédiction...")
plot_prediction_errors(y_true, y_pred)

# 4. Statistiques détaillées
print("\nCalcul des statistiques d'erreur détaillées...")
mae, rmse, r2, mape = calculate_error_statistics(y_true, y_pred)

## 4. Interprétation des Résultats

Analysons les différents indicateurs de validation :

1. **Courbes d'apprentissage** : 
   - Si les courbes d'entraînement et de validation sont proches, le modèle généralise bien
   - Un grand écart indique un surapprentissage

2. **Validation croisée** :
   - La variation des scores entre les folds indique la stabilité du modèle
   - Une grande variation suggère une instabilité

3. **Distribution des erreurs** :
   - Une distribution normale centrée sur 0 est idéale
   - Des biais systématiques sont visibles dans le scatter plot

4. **MAPE** :
   - Donne une idée de l'erreur en pourcentage
   - Plus facile à interpréter pour les utilisateurs métier